In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import csv

In [10]:
driver = webdriver.Chrome()


url_dic={
    'Andhra Pradesh':'https://www.redbus.in/online-booking/apsrtc',
    'Kerala':'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
    'Telangana':'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
    'Goa':'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
    'Himachal Pradesh':'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',
    'Assam':'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
    'Chandigarh':'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
    'Punjab':'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile',
    'Bihar':'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',
    'West Bengal':'https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile',
    'Sowth Bengal':'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',
    'Jammu Kashmir':'https://www.redbus.in/online-booking/jksrtc'
}

bus_data = []
try:
    for state, url in url_dic.items():
        driver.get(url)
        time.sleep(5)

        wait = WebDriverWait(driver, 10)
        page = 1

        all_routes = []
        
        while True:
            print(f"Processing page: {page}")
            try:
                nested_div = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]')))
                route_list = nested_div.find_elements(By.CLASS_NAME, 'route_link')  

                for route in route_list:
                    try:
                        link_element = route.find_element(By.TAG_NAME, 'a')
                        route_name = link_element.get_attribute('title')  
                        route_link = link_element.get_attribute('href')
                        all_routes.append((state, route_name, route_link))
                    except Exception as e:
                        print(f"Error extracting route data: {e}")

                page_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".DC_117_pageTabs")))

                if page >= len(page_buttons): 
                    print("Reached the last page.")
                    break

                next_button = page_buttons[page]
                driver.execute_script("arguments[0].click();", next_button)
                
                page+=1
                time.sleep(2)  

            except Exception as e:
                print(f"Error navigating data on page {page}: {e}")
                break

        for state, route_name, route_link in all_routes:
            try:
                driver.get(route_link)
                time.sleep(3)

                try:
                    view_buses_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".button")))
                    view_buses_button.click() 
                except Exception as e:
                            print(f"no govt busses for this route {e}")

                bus_details = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
                
                for bus in bus_details:
                    try:
                        bus_name = bus.find_element(By.CLASS_NAME, "travels").text
                        bus_type = bus.find_element(By.CLASS_NAME, "bus-type").text
                        departure_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                        duration = bus.find_element(By.CLASS_NAME, "dur").text
                        reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                        
                        try:
                            star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                        except Exception:
                            star_rating  = "N/A"
                        
                        fare = bus.find_element(By.CLASS_NAME, "fare").text.split()[-1]  # Extract numeric fare
       
                        try:
                            seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text.split()[0]
                        except Exception:
                            seats_available = "N/A"

                        bus_data.append({
                            "state": state,
                            "route_name": route_name,
                            "route_link": route_link,
                            "busname": bus_name,
                            "bustype": bus_type,
                            "departing_time": departure_time,
                            "duration": duration,
                            "reaching_time": reaching_time,
                            "star_rating": star_rating,
                            "price": fare,
                            "seats_available": seats_available
                            })
                    except:
                        pass

            except:
                pass

    with open("Redbus1.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=[
            "state", "route_name", "route_link", "busname", "bustype",
            "departing_time", "duration", "reaching_time", "star_rating", "price", "seats_available"
        ])
        writer.writeheader()
        writer.writerows(bus_data)
finally:
    driver.quit()

Processing page: 1
Processing page: 2
Reached the last page.
no govt busses for this route Message: element click intercepted: Element <div class="button view-seats fr">...</div> is not clickable at point (957, 546). Other element would receive the click: <ul class="bus-items">...</ul>
  (Session info: chrome=135.0.7049.96)
Stacktrace:
	GetHandleVerifier [0x00007FF726175355+78597]
	GetHandleVerifier [0x00007FF7261753B0+78688]
	(No symbol) [0x00007FF725F291AA]
	(No symbol) [0x00007FF725F87089]
	(No symbol) [0x00007FF725F84A42]
	(No symbol) [0x00007FF725F81AE1]
	(No symbol) [0x00007FF725F80A42]
	(No symbol) [0x00007FF725F72204]
	(No symbol) [0x00007FF725FA70EA]
	(No symbol) [0x00007FF725F71AB6]
	(No symbol) [0x00007FF725FA7300]
	(No symbol) [0x00007FF725FCF2BB]
	(No symbol) [0x00007FF725FA6EC3]
	(No symbol) [0x00007FF725F703F8]
	(No symbol) [0x00007FF725F71163]
	GetHandleVerifier [0x00007FF72641EF0D+2870973]
	GetHandleVerifier [0x00007FF7264196B8+2848360]
	GetHandleVerifier [0x00007FF726

In [2]:
import pandas as pd
data = pd.read_csv("Redbus.csv")

data['departing_time'] = pd.to_datetime(data['departing_time'], format='%H:%M', errors='coerce').dt.time
data['reaching_time'] = pd.to_datetime(data['reaching_time'], format='%H:%M', errors='coerce').dt.time

if 'route_name' in data.columns:
        split_routes = data['route_name'].str.split(' to ', n=1, expand=True)
        data['from_city'] = split_routes[0].str.strip()
        data['to_city'] = split_routes[1].str.strip() if split_routes.shape[1] > 1 else None

print(data.head())

            state               route_name  \
0  Andhra Pradesh  Hyderabad to Vijayawada   
1  Andhra Pradesh  Hyderabad to Vijayawada   
2  Andhra Pradesh  Hyderabad to Vijayawada   
3  Andhra Pradesh  Hyderabad to Vijayawada   
4  Andhra Pradesh  Hyderabad to Vijayawada   

                                          route_link            busname  \
0  https://www.redbus.in/bus-tickets/hyderabad-to...  IntrCity SmartBus   
1  https://www.redbus.in/bus-tickets/hyderabad-to...           FRESHBUS   
2  https://www.redbus.in/bus-tickets/hyderabad-to...           FRESHBUS   
3  https://www.redbus.in/bus-tickets/hyderabad-to...         AdYolo Bus   
4  https://www.redbus.in/bus-tickets/hyderabad-to...  IntrCity SmartBus   

                      bustype departing_time duration reaching_time  \
0  A/C Seater / Sleeper (2+1)       23:10:00  07h 05m      06:15:00   
1   Electric A/C Seater (2+2)       16:35:00  07h 20m      23:55:00   
2   Electric A/C Seater (2+2)       22:35:00  06h 50m      

In [3]:
data.dropna(inplace=True)

In [4]:
import pymysql

mydb = pymysql.connect(
            host="localhost",
            user="root",
            password="He007#W=Got765ked",  # Update with your database password
            database = "foundation" 
        )
mycursor = mydb.cursor()

In [6]:
mycursor.execute("""CREATE TABLE redbus (
    state VARCHAR(100),from_city VARCHAR(100),to_city VARCHAR(100),
    route_name VARCHAR(255),
    route_link TEXT,
    busname VARCHAR(255),
    bustype VARCHAR(255),
    departing_time TEXT,
    duration VARCHAR(50),
    reaching_time text,
    star_rating DECIMAL(2,1),
    price DECIMAL(10,2),
    seats_available INT
);""")
mydb.commit()

In [7]:
data.columns

Index(['state', 'route_name', 'route_link', 'busname', 'bustype',
       'departing_time', 'duration', 'reaching_time', 'star_rating', 'price',
       'seats_available', 'from_city', 'to_city'],
      dtype='object')

In [9]:
for i,row in data.iterrows():
    insert_query = """insert into redbus(state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,
                       star_rating,price,seats_available,from_city, to_city)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    values = (row['state'],row['route_name'],row['route_link'],row['busname'],row['bustype'],row['departing_time'],row['duration'],
              row['reaching_time'],row['star_rating'],row['price'],row['seats_available'],row['from_city'],row['to_city'])
    mycursor.execute(insert_query,values)
    mydb.commit()

In [14]:
data['bustype'].unique()

array(['A/C Seater / Sleeper (2+1)', 'Electric A/C Seater (2+2)',
       'A/C Seater/Sleeper (2+1)',
       'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'METRO LUXURY A/C',
       'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)',
       'Non A/C Seater / Sleeper (2+1)', 'STAR LINER(NON-AC SLEEPER 2+1)',
       'Express Non AC Seater 2+3', 'A/C Sleeper (2+1)',
       'Bharat Benz A/C Seater /Sleeper (2+1)',
       'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'NON A/C Sleeper (2+1)',
       'Bharat Benz A/C Sleeper (2+1)',
       'Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1)',
       'INDRA(A.C. Seater)', 'Volvo Multi-Axle A/C Sleeper (2+1)',
       'NON A/C Seater (2+2)', 'Swift Deluxe Non AC Air Bus (2+2)',
       'Super Express Non AC Seater Air Bus (2+2)',
       'Super Fast Non AC Seater (2+3)', 'Low Floor AC Seater 2+2',
       'NON A/C Semi Sleeper (2+2)',
       '2+1 Air Suspension A/C Seater / Sleeper',
       'A/C Semi Sleeper (2+2)', 'VE A/C Semi Sleeper (2+2)',
       'SWIFT-GARUDA A/C S